In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization,Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
import holidays


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**I did EDA at other notebook:https://www.kaggle.com/imnaho/eda-tps-mar22**

**Thease days I'm learning LSTM so in this notebook I'll try using LSTM**

# read data

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-mar-2022/sample_submission.csv')
train.shape, test.shape,sample_submission.shape

In [ ]:
cols_with_missing = [col for col in train.columns if train[col].isnull().any()]
print('missing value in train data is:',cols_with_missing)
cols_with_missing = [col for col in test.columns if test[col].isnull().any()]
print('missing value in test data is:',cols_with_missing)

In [ ]:
train.dtypes

In [ ]:
test.dtypes

In [ ]:
a=train.groupby(['x','y','direction']).mean()
a

In [ ]:
train.describe(include='all')

In [ ]:
test.describe(include='all')

at EDA notebook I checked 
* train data and test data has no missing value
* train data has time 1991-04-01 00:00 ~ 1991-09-30 11:40  
* test data has time 1991-09-30 12:00 ~ 1991-09-30 23:40
* columns x,y and direction might be good feature
* week,hour also might be good feature from time column
* there are very little difference at each day

In [ ]:
train

# add some columns

In [ ]:
us_holidays = holidays.US()

for df in [train, test]:
    
    df['date'] = pd.to_datetime(df.time)
    le = LabelEncoder()
    encoded = le.fit_transform(df['direction'].values)
    df['direction']=encoded
    
    df['num_week']=df['date'].dt.week
    df['week']=df['date'].dt.weekday
    
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df["hour"] = df['date'].dt.strftime('%H').astype(int)
    df["minutes"] = df['date'].dt.strftime('%M').astype(int)
    df["num_of_time"] = df["hour"].astype(int)+df["minutes"]/60
    
    train['holyday']=0
    for day in holidays.US(years = 1991).items():
        train.loc[train['date'].astype(str)==day[0].strftime("%Y-%m-%d %H:%M:%S"),'holyday']=1
    train.loc[(train['week']==5)|(train['week']==6),'holyday']=1
    
    
train.head()

In [ ]:
train.groupby('holyday').count()

# prepare training data

In [ ]:
train['compare']=train['num_week'].astype(str)+train['week'].astype(str)+train['hour'].astype(str)

mean=train.groupby(['num_week','week','hour']).mean()
mean=mean.reset_index()
mean['compare']=mean['num_week'].astype(str)+mean['week'].astype(str)+mean['hour'].astype(str)

for mean_list in mean['compare']:
    mean_congestion=mean.loc[mean['compare']==mean_list,'congestion']
    mean_congestion=float(mean_congestion)
    train.loc[train['compare']==mean_list,'mean']=mean_congestion
    
train

In [ ]:
train.mean()

In [ ]:
def target_scale(train_data):
    
    congestion_max=train_data['congestion'].max()
    congestion_min=train_data['congestion'].min()
    congestion_data=(train_data['congestion']-congestion_min)/(congestion_max-congestion_min)
    
    week_max=train_data['week'].max()
    week_min=train_data['week'].min()
    week_data=(train_data['week']-week_min)/(week_max-week_min)
    
    mean_max=train_data['mean'].max()
    mean_min=train_data['mean'].min()
    mean_data=(train_data['mean']-mean_min)/(mean_max-mean_min)
    
    return congestion_data,week_data,mean_data,congestion_max,congestion_min

In [ ]:
def make_dataset(low_data):

    data, target = [], []
    maxlen = 7

    for i in range(len(low_data)-maxlen):
        data.append(low_data[i:i + maxlen])
        target.append(low_data.loc[i + maxlen,'congestion'])

    re_data = np.array(data).reshape(len(data), maxlen, 4)
    re_target = np.array(target).reshape(len(data),1)

    return re_data, re_target

In [ ]:
def make_test_dataset(low_data):

    data, target = [], []
    maxlen = 7

    data.append(low_data)
    re_data = np.array(data).reshape(len(data), maxlen, 4)
    
    return re_data

In [ ]:
def make_lstm_model(g,h,g_test,congestion_max,congestion_min):
    n_hidden = 200
    
    model=None
    model = keras.Sequential()
    model = Sequential()
    model.add(LSTM(n_hidden, batch_input_shape=(None,g.shape[1], g.shape[2]),activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(2))
    model.add(Activation("relu"))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    model.add(Activation("linear"))
    optimizer = Adam(lr=0.001)
    model.compile(loss="mean_squared_error", optimizer=optimizer)
    
    #early_stopping = EarlyStopping(monitor='val_loss', mode='auto', min_delta=0.0001)
    history=model.fit(g, h,
                      batch_size=1,
                      epochs=25,
                      #validation_split=0.1,
                      #callbacks=[early_stopping],
                      verbose=0,
                      shuffle=False
                     )
    
    a=model.predict(g_test)
    a=a*(congestion_max-congestion_min)+congestion_min
    a=np.round(a)
    return a

# fitting model and make prediction

In [ ]:
df_prediction=pd.DataFrame()

df_list=[]
score_sum = 0

a=train.groupby('num_of_time').sum()
time_list=a.index
time_predict_list = [i for i in time_list if i>=12]


for i in range(3):
    
    for j in range(4):
        
        for k in range(8):
            
            aa=None
            aa=train[train['x']==i]
            aa=aa[aa['y']==j]
            aa=aa[aa['direction']==k]
            
            b=None
            b=test[test['x']==i]
            b=b[b['y']==j]
            b=b[b['direction']==k]
            
            if len(aa)==0:
                continue
            
            pre_model=None
            pre_score=0
            
            target_,week,mean,congestion_max,congestion_min=target_scale(aa)
            aa['congestion']=target_
            aa['week']=week
            aa['mean']=mean
            #print(i,j,k)
            
            for time in time_predict_list:
                
                traindata=pd.DataFrame()
                str_=str(time)
                
                congestion=aa.loc[aa['num_of_time']==time,['congestion','week','mean','holyday']]
                
                congestion=congestion.reset_index()
                congestion=congestion.drop('index',axis=1)
                
                traindata=congestion.copy()
                
                prediction_id=b.loc[b['num_of_time']==time,['row_id']]
                
                predict_target=None
                predict_target=traindata.tail(7)
                predict_target=predict_target.reset_index()
                predict_target=predict_target.drop('index',axis=1)
                
                g=None
                h=None
                g_test=None
                
                g, h = make_dataset(traindata)
                g_test=make_test_dataset(predict_target)
                
                prediction = make_lstm_model(g,h,g_test,congestion_max,congestion_min)
                #print(time,prediction)
                
                sample_submission.loc[sample_submission['row_id']==int(prediction_id.row_id),'congestion']=prediction

# check submit data situation and create csv data

In [ ]:
sample_submission

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(sample_submission['congestion'])

In [ ]:
sample_submission.to_csv('submission.csv', index=False)

# conclusion

in this notebook I tried using LSTM model 

honestly, preparing data for LSTM model takes more time and effort than using xgbregressor like this notebook I did https://www.kaggle.com/code/imnaho/tps22-mar-predict-with-xgbregressor

but for me, it is good experience and I'll try improve score!!